### Importing Libraries


In [1]:
import cv2
import numpy as np
import os
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report

### Loading Spectrograms

In [13]:
import os
x = [name for name in os.listdir('dataset/Spectrograms/')] 
print(x)
files={}
label_dic={}
cnt = 0

for i in x:
    files[i]= list(data_dir.glob(f'{i}/*'))
    label_dic[i]= cnt
    cnt = cnt+1
    
print(label_dic)

['CommonCuckoo', 'CommonKestrel', 'CommonRingedPlover', 'LittleOwl', 'NorthernRaven']
{'CommonCuckoo': 0, 'CommonKestrel': 1, 'CommonRingedPlover': 2, 'LittleOwl': 3, 'NorthernRaven': 4}


In [14]:
lt = [name for name in os.listdir('dataset/Spectrograms/')]
image_files = pathlib.Path('dataset/Spectrograms/')
gg = {}
for i in lt:
    gg[i]=list(image_files.glob(f'{i}/*'))

In [15]:
len(gg)

5

In [16]:
x, y = [], []

for bird , images in gg.items():
    for image in images:
        img = cv2.imread(str(image))
        x.append(img)
        y.append(label_dic[bird])

### Spliting the dataset

In [17]:
x = np.array(x)
y = np.array(y)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
model = keras.Sequential([
keras.layers.Conv2D(input_shape=(480,640,3),filters=32, kernel_size=3, strides=2,activation='relu'), 
keras.layers.BatchNormalization(), 
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'), 
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(),

keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, activation='relu'), 
keras.layers.BatchNormalization(),
keras.layers.BatchNormalization(),
keras.layers.Flatten(),
keras.layers.Dense (256, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.4),
keras.layers.Dense (5, activation="softmax"),
])

model.compile(optimizer= 'adam',
              loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])
with tf.device('/CPU:0'):
    model.fit(x_train, y_train, epochs=8)

Epoch 1/8
26/26 [==============================] - 33s 1s/step - loss: 1.2723 - accuracy: 0.6219
Epoch 2/8
26/26 [==============================] - 33s 1s/step - loss: 0.7640 - accuracy: 0.7338
Epoch 3/8
26/26 [==============================] - 32s 1s/step - loss: 0.6591 - accuracy: 0.7562
Epoch 4/8
26/26 [==============================] - 33s 1s/step - loss: 0.6100 - accuracy: 0.7811
Epoch 5/8
26/26 [==============================] - 34s 1s/step - loss: 0.6055 - accuracy: 0.7799
Epoch 6/8
26/26 [==============================] - 34s 1s/step - loss: 0.5087 - accuracy: 0.7998
Epoch 7/8
26/26 [==============================] - 33s 1s/step - loss: 0.4617 - accuracy: 0.8358
Epoch 8/8
26/26 [==============================] - 32s 1s/step - loss: 0.3406 - accuracy: 0.8856


In [29]:
with tf.device('/CPU:0'):
    model.evaluate(x_test, y_test)

7/7 [==============================] - 2s 300ms/step - loss: 0.5866 - accuracy: 0.7811


### Save the model

In [ ]:
model.save('model.keras')

### Calculating the Classification Report

In [31]:
with tf.device('/CPU: 0'):
    y_pre = model.predict(x_test)

7/7 [==============================] - 2s 324ms/step


In [32]:
y_pred = [np.argmax(i)for i in y_pre]

In [33]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84        32
           1       0.82      0.80      0.81        50
           2       0.89      0.80      0.84        50
           3       0.50      0.69      0.58        26
           4       0.82      0.74      0.78        43

    accuracy                           0.78       201
   macro avg       0.77      0.78      0.77       201
weighted avg       0.80      0.78      0.79       201

